# Sentiment Prediction

In this last part of the hackathon, we will train a small deep learning model on the IMDB Sentiment dataset.
This dataset contains review texts along with a labelled "positive" or "negative" sentiment.
We will preprocess the data using Polars, which you will implement.
We will then train and evaluate a neural network that can predict a sentiment for an arbitrary input text!

In [9]:
import joblib
import polars as pl
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Load the data
First of all we will load the data.
It is always a good idea to visualise a portion of the data and do a few sanity checks before starting the preprocessing.

### Exercise 5.1
#### Exercise 5.1.1
Load the data using polars.
Since in this case we are going to process all the data at once, we can read it in directly rather than scanning and collecting separate parts in our analysis.
However, a golden tip is that it is still always a good idea convert the DataFrame to a LazyFrame to allow the optimization engine to optimize all the queries we define subsequently.

#### Exercise 5.1.2
Using only Polars, show the number of records in the dataset, (separately) visualise the first few records and the last few records of the data and show the counts per sentiment value.

## Preprocessing
In case of a larger dataset, it may be a good idea to use the `tf.Dataset` interface to allow preprocessing portions of the data in parallel with training on the data that has already been preprocessed, which can be accomplished through the `map()` function.
In this case, our dataset is relatively small and we can do the full preprocessing beforehand.

First off, for the training procedure we will want to use integer values for our labels.
In the case of binary values, mapping to 0 and 1 is most commonly used.

### Exercise 5.2
Convert the sentiment column in the data frame by mapping positive reviews to the value 1 and negative reviews to the value 0.
Display the first few records to assure that the conversion was done correctly, then display the value counts to assure that the column now only consists of 0 and 1 values and that the counts are identical to before.

### Exercise 5.3
Split the data into two distinct sets of train and test data respectively.
Use the imported `train_test_split` function.
Visualise the output shapes to ensure that the ratio between the two is as expected.


## Tokenize the review texts
An important concept in natural language processing is tokenization.
As individual characters have little meaning and would yield an extremely high-dimensional search space, texts are instead tokenized into common sequences of characters.
For LLM's such as GPT, the tokenization is nowadays commonly done character-based.
However, especially for smaller models it is a good idea to use whole words as tokens.

### Exercise 5.4
Use the keras Tokenizer (already imported for your convenience) to tokenize our input data.
#### Exercise 5.4.1
Instantiate the Tokenizer and fit it on the review texts of our **training data** using the `fit_on_texts()` method.
In this manner, the Tokenizer will determine how to tokenize data based on the provided fitting data.
You can directly pass in the Polars Series object, as it implements the required iterator for said method.

In [15]:
tokenizer = Tokenizer(num_words=5000)

#### Exercise 5.4.2
Subsequently, convert the reviews from our training and test data to create the final input data (X) to use for training and testing using the `texts_to_sequences` method on the just created instance.
Display the resulting arrays to ensure that the text was tokenized and to get a rough idea of the input for our model.

#### Exercise 5.4.3
Convert our training and test labels to numpy arrays.
Again inspect both arrays to ensure the content is roughly as expected.

## Defining and compiling the model
Here we build and train the model.
The model we will be using is a small model, and it looks simple because Keras defined a great deal for us.
Although the model itself is outside of the scope of this hackathon, of course I'll give a brief explanation of the different parts.
The Embedding layer defines takes the sparse input data and embeds it into a lower-dimensional space that is easier for our model to work with.
The output of this layer is passed into the LSTM (Long Short-Term Memory) submodel, which is a model consisting of multiple layers internally, designed to learn how to relate sequential data (e.g. words in a sentence).
Finally, the output of the LSTM is passed into a Dense layer with a Sigmoid activation function.
This layer will give a continuous output between 0 and 1.
During the training phase, we will use the loss function ("binary cross-entropy" as defined below) to steer the model towards predicting a score close to 0 for the labels we defined as 0, and 1 for the labels we defined as 1.


In [ ]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=250))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

model.summary()

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

## Train the model

In [ ]:
model.fit(train_x, train_y, epochs=5, batch_size=64, validation_split=0.2)

## Save the model and tokenizer
We save the model and tokenizer to be able to load them for a different purpose or in a later phase.
For the remainder of this notebook, however, we will keep using the instances we have here.

In [ ]:
model.save("model.h5")
joblib.dump(tokenizer, "tokenizer.pkl")

## Evaluate the model

In [ ]:
model.evaluate(test_x, test_y)

## Use the model to predict the sentiment for a review
### Exercise 5.5
Define a function that takes a review text as input and uses the tokenizer to convert the text to a token sequence, pad it similarly to the input data and finally provide it as input for our model.
Then read the output and convert it to a "positive" or "negative" string.

In [23]:
def predict_sentiment_for_review(review):
    # Implement me
    return None

In [ ]:
predict_sentiment_for_review("I'd rather watch paint dry")

In [ ]:
predict_sentiment_for_review("The best movie I've ever seen")

In [ ]:
predict_sentiment_for_review("Absolutely stunning!")

In [ ]:
predict_sentiment_for_review("Even The Titanic was better...")

In [ ]:
predict_sentiment_for_review("sucks")